In [3]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import asyncio
import csv
import random

from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

import numpy as np
import pandas as pd

In [21]:
CHROME_PATH = ''
PATH_TO_SAVE_COMMENTS = ''
PATH_TO_SAVE_LINKS = ''
PATH_TO_SAVE_RESTAURANTS = ''

MAX_LINKS = 10

df = pd.DataFrame({"quote":[], "content":[],"rating":[],"date":[],"place":[]})
links_list = ""
all_data = []
restaurant_data = []
place_info = ""
list_seen = []

main_url = 'https://pl.tripadvisor.com/Restaurants-g274772-Krakow_Lesser_Poland_Province_Southern_Poland.html'
url_list = []

#collector = LinkCollector(main_url)
#collector.setup_link_collector()

In [22]:
def setup_scrapper_and_start_process():
    scrapper = SinglePageScrapper()
    links_list = pd.read_csv(PATH_TO_SAVE_LINKS)['url'].tolist()
    links_list = list(reversed(links_list))
    
    while len(links_list) > 0 :
        scrapper.setup_work(links_list[0])
        links_list = links_list[1:]
        
    scrapper.driver.quit()

In [23]:
def process_row(row):
    all_data.append(row)
    
def save_dataframe():
    df = pd.DataFrame(all_data)
    df.to_csv(PATH_TO_SAVE_COMMENTS, index = False, header=True)
    
def save_restaurant_info(row):
    restaurant_data.append(row)
    
def save_places():
    df = pd.DataFrame(restaurant_data)
    df.to_csv(+PATH_TO_SAVE_RESTAURANTS, index = False, header=True)
    
def save_links():
    df = pd.DataFrame(url_list)
    df.to_csv(PATH_TO_SAVE_LINKS, index = False, header=True)

In [7]:
class SinglePageScrapper:
    def __init__(self):
        self.driver = webdriver.Chrome(CHROME_PATH)
        self.wait = WebDriverWait(self.driver, 10)
        self.height = ""
        self.url = ""
        self.soup = ""
        self.reviev_links = []
        self.place_name = ""
        
    def end_work(self):
        save_dataframe()
        save_places()
        
    def setup_work(self,url):
        self.url = url
        self.driver.get(self.url)
        source = self.driver.page_source
        self.soup = BeautifulSoup(source, 'html.parser')
        
        try:
            m_place_name = self.soup.find('h1',{'class':'_3a1XQ88S'}).getText()
            self.place_name = m_place_name
        except: print("error")
        
        self.scroll_down()
        self.get_review_links()
        self.start_scrapping()
        
    def get_place_info(self):
        try:
            m_place_name = self.soup.find('h1',{'class':'_3a1XQ88S'}).getText()
            self.place_name = m_place_name
        except: print("error")
        
        place_info = self.soup.find('span',{'class':'_13OzAOXO _34GKdBMV'})
        m_prices_info = len(place_info.find_all('a')[0].getText())
        
        detail_info = self.soup.find('div',{'class':'_3UjHBXYa'})
        detail_info = detail_info.find_all('div',{'class':'_1XLfiSsv'})
        
        m_cuisine_type = detail_info[1].getText()
        m_others = detail_info[2].getText()
            
        num_rec = self.soup.find('div',{'class':'Ct2OcWS4 _3nlSp84z'})
        
        m_num_rec = num_rec.find('a',{"class":'_10Iv7dOs'}).getText()
        
        m_avg_rec = self.soup.find('span',{'class':'r2Cf69qf'}).getText()
        
        m_place = self.soup.find('span',{'class':'_2saB_OSe _1OBMr94N'}).find_all('div')[0].getText()
        
        m_obj = {'name':self.place_name,'prices':m_prices_info,'cuisine':m_cuisine_type,'others':m_others,'num_rec':m_num_rec,'avg_rec':m_avg_rec,'location':m_place}
        
        save_restaurant_info(m_obj)
        
    def scroll_down(self):
        speed = 20
        current_scroll_position = 0
        new_height = 1
        while current_scroll_position <= new_height:
            current_scroll_position += speed
            self.driver.execute_script("window.scrollTo(0, {});".format(current_scroll_position))
            new_height = self.driver.execute_script("return document.body.scrollHeight")
            
    def get_review_links(self):
        comment_elements = self.soup.find("div",{"class":"pageNumbers"}).find_all('a')
        for ele in comment_elements:
            if ele.has_attr('href'):
                a_url = "https://pl.tripadvisor.com"+ele.attrs['href']
                self.reviev_links.append(a_url)
            
    def start_scrapping(self):
        self.get_place_info()
        
        for ele in self.reviev_links:
            self.load_info(ele)
            
        self.end_work()
            
    def get_rating(self,m_str): 
        m_str = ''.join(reversed(m_str))
        return m_str[1]
            
    def load_info(self, link):
        self.driver.get(link)
        source = self.driver.page_source
        self.soup = BeautifulSoup(source, 'html.parser')
        
        r_divs = self.soup.find_all("div",{"class":"review-container"})
        
        r_selenium_divs = self.driver.find_elements_by_class_name('review-container')          
        
        for ele in r_divs:
            m_index = r_divs.index(ele)
            
            try:
                click_p = r_selenium_divs[m_index].find_element_by_xpath("//p[contains(@class,'partial_entry')]")
                click_span = click_p.find_element_by_xpath("//span[contains(@class,'taLnk ulBlueLinks')]") 
                ActionChains(self.driver).move_to_element(click_span).click().perform()
            except:
                print("No 'more' span!") 
            
            m_quote = ele.find("div",{"class":"quote"}).getText()
            m_content = ele.find("p",{"class":"partial_entry"}).getText()
            m_stay_date = ""

            try:
                stay_date_div = ele.find("div",{"class":"prw_rup prw_reviews_stay_date_hsx"})
                stay_date_list = stay_date_div.getText().split(" ")
                stay_date_list = list(reversed(stay_date_list))
                m_stay_date = stay_date_list[1]+' '+stay_date_list[0]
            except: print('error')

            rating_container = self.soup.find("div",{"class":"ui_column is-9"}).find_all("span")[0]
            m_rating = self.get_rating(rating_container['class'][1]) 
            
            ele_html = ele.encode_contents()
            
            temp_list = {"quote":m_quote, "content":m_content,"rating":m_rating,"date":m_stay_date, "place":self.place_name}
            print(temp_list)
            process_row(temp_list)           

In [24]:
class LinkCollector:
    def __init__(self,url):
        self.driver = webdriver.Chrome(CHROME_PATH)
        self.wait = WebDriverWait(self.driver, 10)
        self.height = ""
        self.url = url
        self.soup = ""
        self.reviev_links = []
        self.place_name = []
        global url_list
              
    def setup_link_collector(self):
        self.scroll_down()
        self.driver.get(self.url)
        source = self.driver.page_source
        self.soup = BeautifulSoup(source,'html.parser')
        repeated = False
        next_span_disabled='a'
        
        try:
            next_span_disabled = self.driver.find_element_by_xpath("//span[contains(@class,'nav next disabled')]")
        except: print("not yet!")
        
        while next_span_disabled == 'a' or len(url_list) < MAX_LINKS :
            self.scroll_down()
            link_list = self.soup.find_all('div',{'class':'_1kNOY9zw'})
            
            try:
                next_btn = self.driver.find_element_by_xpath("//a[contains(@class,'nav next rndBtn ui_button primary taLnk')]")
            except: print("end of scrapping!")
                
            for ele in link_list:
                n_link = "https://pl.tripadvisor.com"+ele.find('a',{'class':'_15_ydu6b'}).attrs['href']
                for i in url_list: 
                    if i['url'] == n_link: 
                        repeated = True
                    
                if not repeated: url_list.append({"url":n_link, "visited": False})
            
            try:
                ActionChains(self.driver).move_to_element(next_btn).click().perform()
            except: print("end of scrapping!")
                    
            try:
                next_span_disabled = self.driver.find_element_by_xpath("//span[contains(@class,'nav next disabled')]")
            except: print("not yet!")
            
        save_links()
        setup_scrapper_and_start_process()
        
    def scroll_down(self):
        speed = 40
        current_scroll_position = 0
        new_height = 1
        while current_scroll_position <= new_height:
            current_scroll_position += speed
            self.driver.execute_script("window.scrollTo(0, {});".format(current_scroll_position))
            new_height = self.driver.execute_script("return document.body.scrollHeight")